In [1]:
import numpy as np
import torch
from torch import nn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:

class RidgeRegression(nn.Module):
    """
    PyTorch implementation of Ridge Regression
    """
    def __init__(self, input_dim):
        super(RidgeRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1, bias=False)
        
    def forward(self, x):
        return self.linear(x)
    
def fisher_z_transform(r):
    """
    Apply Fisher Z transformation to correlation coefficient
    """
    # Clip r to avoid numerical issues
    r = np.clip(r, -0.999999, 0.999999)
    return 0.5 * np.log((1 + r) / (1 - r))

def compute_correlation(y_true, y_pred):
    """
    Compute Pearson correlation and its Fisher Z transformation
    """
    # Convert to numpy if tensors
    if torch.is_tensor(y_true):
        y_true = y_true.cpu().numpy()
    if torch.is_tensor(y_pred):
        y_pred = y_pred.cpu().numpy()
    
    # Compute correlation
    correlation = np.corrcoef(y_true, y_pred)[0, 1]
    
    # Apply Fisher Z transformation
    z_score = fisher_z_transform(correlation)
    
    return correlation, z_score

def nested_loocv_ridge_cuda(X_train, y_train, alphas, device):
    """
    GPU-accelerated nested leave-one-out cross-validation
    """
    n_samples = len(y_train)
    mse_alphas = torch.zeros(len(alphas), device=device)
    
    # Convert to PyTorch tensors
    X_train = torch.FloatTensor(X_train).to(device)
    y_train = torch.FloatTensor(y_train).to(device)
    
    for i in range(n_samples):
        # Create LOO split
        mask = torch.ones(n_samples, dtype=torch.bool, device=device)
        mask[i] = False
        X_nested_train = X_train[mask]
        y_nested_train = y_train[mask]
        X_nested_val = X_train[~mask]
        y_nested_val = y_train[~mask]
        
        for j, alpha in enumerate(alphas):
            # Solve ridge regression analytically on GPU
            XtX = torch.mm(X_nested_train.t(), X_nested_train)
            reg_term = alpha * torch.eye(X_nested_train.shape[1], device=device)
            Xty = torch.mm(X_nested_train.t(), y_nested_train.unsqueeze(1))
            weights = torch.linalg.solve(XtX + reg_term, Xty)
            
            # Compute predictions and MSE
            y_pred = torch.mm(X_nested_val, weights)
            mse = torch.nn.functional.mse_loss(y_pred, y_nested_val.unsqueeze(1))
            mse_alphas[j] += mse
    
    mse_alphas /= n_samples
    best_alpha = alphas[torch.argmin(mse_alphas)]
    
    return best_alpha.item()

def time_series_ridge_cv_cuda(X, y, n_splits=5, device='cuda'):
    """
    GPU-accelerated ridge regression with nested cross-validation and correlation analysis
    """
    # Ensure y is 1D
    y = y.ravel()
    
    # Create logarithmically spaced alphas
    alphas = torch.logspace(-1, 8, 10).to(device)
    
    # Initialize TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    # Initialize arrays to store results
    predictions = np.zeros_like(y)
    coefficients = []
    alphas_selected = []
    correlations = []
    z_scores = []
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Perform cross-validation
    for fold, (train_idx, test_idx) in enumerate(tscv.split(X_scaled)):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        # Find best alpha using nested LOOCV
        best_alpha = nested_loocv_ridge_cuda(X_train, y_train, alphas, device)
        alphas_selected.append(best_alpha)
        
        # Convert to PyTorch tensors
        X_train_tensor = torch.FloatTensor(X_train).to(device)
        y_train_tensor = torch.FloatTensor(y_train).to(device)
        X_test_tensor = torch.FloatTensor(X_test).to(device)
        
        # Solve ridge regression analytically on GPU
        XtX = torch.mm(X_train_tensor.t(), X_train_tensor)
        reg_term = best_alpha * torch.eye(X_train.shape[1], device=device)
        Xty = torch.mm(X_train_tensor.t(), y_train_tensor.unsqueeze(1))
        weights = torch.linalg.solve(XtX + reg_term, Xty)
        
        # Generate predictions
        fold_predictions = torch.mm(X_test_tensor, weights).cpu().numpy().ravel()
        predictions[test_idx] = fold_predictions
        coefficients.append(weights.cpu().numpy().ravel())
        
        # Compute correlation and Fisher Z score for this fold
        fold_corr, fold_z = compute_correlation(y_test, fold_predictions)
        correlations.append(fold_corr)
        z_scores.append(fold_z)
        
        print(f"Fold {fold + 1}:")
        print(f"  Selected alpha = {best_alpha:.2e}")
        print(f"  Correlation = {fold_corr:.4f}")
        print(f"  Fisher Z = {fold_z:.4f}")
    
    # Calculate average correlation and Z-score
    mean_correlation = np.mean(correlations)
    mean_z = np.mean(z_scores)
    std_z = np.std(z_scores, ddof=1)  # Sample standard deviation
    
    # Calculate standard error of mean Z
    se_z = std_z / np.sqrt(len(z_scores))
    
    results = {
        'predictions': predictions,
        'coefficients': coefficients,
        'alphas': alphas_selected,
        'correlations': correlations,
        'z_scores': z_scores,
        'mean_correlation': mean_correlation,
        'mean_z': mean_z,
        'std_z': std_z,
        'se_z': se_z
    }
    
    return results


In [3]:

# Example usage
if __name__ == "__main__":
    # Check if CUDA is available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Generate sample data
    np.random.seed(42)
    X = np.random.randn(4000, 20)  # Design matrix
    true_weights = np.random.randn(20)
    y = X @ true_weights + np.random.randn(4000) * 0.1  # Neuroimaging data with noise
    
    # Run the analysis
    results = time_series_ridge_cv_cuda(X, y, device=device)
    
    # Print summary statistics
    print("\nSummary Statistics:")
    print(f"Mean Correlation: {results['mean_correlation']:.4f}")
    print(f"Mean Fisher Z: {results['mean_z']:.4f}")
    print(f"Standard Error of Z: {results['se_z']:.4f}")
    
    # Calculate 95% confidence interval for Z
    ci_lower = results['mean_z'] - 1.96 * results['se_z']
    ci_upper = results['mean_z'] + 1.96 * results['se_z']
    print(f"95% CI for Fisher Z: [{ci_lower:.4f}, {ci_upper:.4f}]")

Using device: cuda
Fold 1: Selected alpha = 1.00e-01
Fold 2: Selected alpha = 1.00e-01
Fold 3: Selected alpha = 1.00e-01
Fold 4: Selected alpha = 1.00e-01
Fold 5: Selected alpha = 1.00e-01

Overall MSE: 4.6142


In [8]:
coefficients

[array([ 0.68238485, -0.2980126 , -0.06943108, -0.17886233,  0.5160941 ,
         0.276757  ,  1.9213984 , -0.38335347,  1.7730253 , -1.808866  ,
         0.50812405, -1.6004441 , -1.013356  , -0.97546816, -1.0550156 ,
        -0.61444616,  2.2170784 , -0.8808186 , -1.6139016 ,  0.49008888],
       dtype=float32),
 array([ 0.6792875 , -0.29394433, -0.06648966, -0.17559113,  0.51445717,
         0.28188184,  1.9177616 , -0.3843823 ,  1.7748672 , -1.8126826 ,
         0.50592977, -1.5973314 , -1.0138994 , -0.9751062 , -1.0565802 ,
        -0.60585177,  2.2155428 , -0.88102376, -1.6150471 ,  0.4866335 ],
       dtype=float32),
 array([ 0.68      , -0.29448393, -0.06751471, -0.17617416,  0.5129547 ,
         0.28336236,  1.9188166 , -0.38223845,  1.7725924 , -1.8132793 ,
         0.5040322 , -1.5984639 , -1.0132424 , -0.9745856 , -1.0518954 ,
        -0.6084133 ,  2.21333   , -0.881147  , -1.6154313 ,  0.4861626 ],
       dtype=float32),
 array([ 0.6790763 , -0.29564774, -0.06913804, -0.17